In [2]:
API_Key = "KLRsc5Lnysc8hE1ZVv26xrXFU6DDUp9zMYnDqXI0TfV5yOzZvvPTi2lQNMsPuIGV"
Secret_Key = "It42rMIfL3bYGQPBMvx9YPkpD2o9R6i7MiuqUvXu8KxIdDIh2iUBoUCxyAFkBxdl"


#tempo


API_Key= '2JGDmHE53G5ADCRt4WOWdkbnTtZdd5HLuRLdjsWzAxG1chUufEf8IB9nB1BdDdAb'
Secret_Key= 'KinhlSfC31xPHZ1Nr2G5PT0VNKF0piE5mkoM5KyzUfXS2GZlOIeTkfoDu3HB91R8'

In [3]:
from binance.client import Client
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt

# Initialize client (fill in your API keys)
client = Client(api_key=API_Key, api_secret=Secret_Key)

def fetch_futures_klines(symbol: str, start_date: str | None, interval: str, end_date: str | None = None):

    klines = []
    limit = 1500  # max per request

    # Convert to ms since Binance uses ms
    if start_date is None:
        # Use earliest valid timestamp for the given interval, if available
        try:
            start_ts = int(client._get_earliest_valid_timestamp(symbol, interval))
        except Exception:
            # Fallback: pull last ~2 batches worth of data if earliest not available
        
            now_ms = int(datetime.datetime.utcnow().timestamp() * 1000)
            start_ts = now_ms - (interval_to_milliseconds(interval) * limit * 2)
    else:
        start_ts = int(date_to_milliseconds(start_date))

    end_ts = int(datetime.datetime.utcnow().timestamp() * 1000) if end_date is None else int(date_to_milliseconds(end_date))

    current_start = start_ts

    while current_start < end_ts:
        # Fetch a batch of klines
        batch = client.futures_klines(
            symbol=symbol,
            interval=interval,
            startTime=current_start,
            endTime=end_ts,
            limit=limit
        )

        if not batch:
            print("No more data returned from Binance.")
            break

        klines.extend(batch)

        # Move start to next interval after the newest kline
        newest_open_time = batch[-1][0]
        current_start = newest_open_time + interval_to_milliseconds(interval)

        # Safety sleep to avoid rate limits
        time.sleep(0.3)

        # Stop if we've reached or passed end
        if newest_open_time >= end_ts:
            break

    return klines

In [5]:
data_pd.to_csv("data.csv", index=False)

In [4]:
data = fetch_futures_klines("DOGEUSDT", "2025-10-01", interval="1m")
data_pd= pd.DataFrame(data, columns=[
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "number_of_trades",
    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignored"
])
data_pd['open_time'] = pd.to_datetime(data_pd['open_time'], unit='ms')
data_pd['close_time'] = pd.to_datetime(data_pd['close_time'], unit='ms')
data_pd = data_pd.drop(columns=['ignored'])

# Compute EMA indicators on close price
# Ensure numeric dtype for price/volume columns
cols_to_num = ["open", "high", "low", "close", "volume"]
for c in cols_to_num:
    data_pd[c] = pd.to_numeric(data_pd[c], errors="coerce")

# Helper to add EMA columns
def add_ema_indicators(df, price_col="close", spans=(9, 20, 50, 200)):
    for span in spans:
        df[f"EMA_{span}"] = df[price_col].ewm(span=span, adjust=False).mean()
    return df

data_pd = add_ema_indicators(data_pd)

data_pd['cossing_9_20'] = data_pd['EMA_9'] > data_pd['EMA_20']
data_pd['cossing_20_50'] = data_pd['EMA_20'] > data_pd['EMA_50']
data_pd['cossing_50_200'] = data_pd['EMA_50'] > data_pd['EMA_200']

data_pd['uptrend'] = (data_pd['cossing_9_20']) & (data_pd['cossing_20_50'])
data_pd['downtrend'] = (~data_pd['cossing_9_20']) & (~data_pd['cossing_20_50'])



No more data returned from Binance.


In [ ]:
data_pd['dif_close_high'] =  data_pd['high'] - data_pd['close']
data_pd['dif_close_low'] = data_pd['close'] - data_pd['low']

rng = data_pd['high'] - data_pd['low']
pos = ((data_pd['close'] - data_pd['low']) / rng).where(rng != 0)  # avoid div-by-zero
data_pd['close_hi_lo_scale'] = (pos * 2 - 1).fillna(0).clip(-1, 1)

# Optional: simple flag for which side it's closer to
data_pd['closer_to_high'] = (data_pd['high'] - data_pd['close']) < (data_pd['close'] - data_pd['low'])



In [ ]:
list_up_down =[]
for i in range(len(data_pd)-1-30):
    if i>10:
        fila_1 = data_pd['close_hi_lo_scale'].iloc[i]
        fila_2 = data_pd['close_hi_lo_scale'].iloc[i-1]
        if fila_1 >0.6 and fila_2 >0.6 and data_pd['uptrend'].iloc[i]==True:
            data_pd[i-10:i+30].plot(x='close_time', y=['close','high','low'], figsize=(15, 7))
            percentage_up = round(((data_pd['high'].iloc[i+1:i+6].max() - data_pd['close'].iloc[i])/data_pd['close'].iloc[i])*100,4)
            percentage_down = round(((data_pd['close'].iloc[i] - data_pd['low'].iloc[i+1:i+6].min())/data_pd['close'].iloc[i])*100,4)
            list_up_down.append([percentage_up,percentage_down])
            #print(percentage_up, "----", percentage_down)
            #plt.axvline(x=data_pd['close_time'].iloc[i], color='red', linestyle='--')
            #plt.show()

    #if i>500:
       # break

In [ ]:
def place_futures_trade_pct(
    side: str,
    symbol: str,
    usdt_to_use: float,
    leverage: int,
    take_profit_pct: float = None,
    stop_loss_pct: float = None
):
    """
    Place a market futures order with TP/SL in percent.
    - side: "BUY" or "SELL"
    - symbol: e.g "DOGEUSDT"
    - usdt_to_use: e.g 5
    - leverage: e.g 10, 30, 50
    - take_profit_pct: e.g 10 for +10% profit target
    - stop_loss_pct: e.g 5 for -5% stop loss
    """

    # 1) Set leverage
    client.futures_change_leverage(symbol=symbol, leverage=leverage)

    # 2) Get current mark price
    price_info = client.futures_mark_price(symbol=symbol)
    entry_price = float(price_info["markPrice"])
    print(f"Entry price: {entry_price}")

    # 3) Compute quantity
    notional = usdt_to_use * leverage
    raw_qty = notional / entry_price

    # Get step size to round quantity
    info = client.futures_exchange_info()
    sym_info = next(s for s in info["symbols"] if s["symbol"] == symbol)
    step_size = float([f["stepSize"] for f in sym_info["filters"] if f["filterType"]=="LOT_SIZE"][0])
    qty = math.floor(raw_qty / step_size) * step_size

    print(f"Qty (rounded): {qty}")

    # 4) Market order
    order = client.futures_create_order(
        symbol=symbol,
        side=side,
        type="MARKET",
        quantity=qty
    )
    print("Market order placed:", order)

    # Get tick size for price rounding
    tick_size = float([f["tickSize"] for f in sym_info["filters"] if f["filterType"]=="PRICE_FILTER"][0])

    # 5) Calculate tp/sl prices
    if take_profit_pct is not None:
        if side.upper() == "BUY":
            tp_price = entry_price * (1 + take_profit_pct/100)
        else:
            tp_price = entry_price * (1 - take_profit_pct/100)
        tp_price = math.floor(tp_price / tick_size) * tick_size

        tp_side = "SELL" if side.upper()=="BUY" else "BUY"
        tp_order = client.futures_create_order(
            symbol=symbol,
            side=tp_side,
            type="TAKE_PROFIT_MARKET",
            stopPrice=tp_price,
            closePosition=True
        )
        print(f"Take Profit @{take_profit_pct}% => {tp_price}:", tp_order)

    if stop_loss_pct is not None:
        if side.upper() == "BUY":
            sl_price = entry_price * (1 - stop_loss_pct/100)
        else:
            sl_price = entry_price * (1 + stop_loss_pct/100)
        sl_price = math.ceil(sl_price / tick_size) * tick_size

        sl_side = "SELL" if side.upper()=="BUY" else "BUY"
        sl_order = client.futures_create_order(
            symbol=symbol,
            side=sl_side,
            type="STOP_MARKET",
            stopPrice=sl_price,
            closePosition=True
        )
        print(f"Stop Loss @{stop_loss_pct}% => {sl_price}:", sl_order)

    # 6) Fee estimate (taker)
    fee_pct = 0.0005  # approx 0.05%
    fee_est = notional * fee_pct * 2
    print(f"Estimated fees (entry+exit): ~{fee_est:.6f} USDT")

    return {
        "entry_price": entry_price,
        "qty": qty,
        "estimated_fees": fee_est
    }

In [ ]:
place_futures_trade_pct(
    side="BUY",
    symbol="DOGEUSDT",
    usdt_to_use=2,
    leverage=30,
    take_profit_pct=7,   # ~10% target
    stop_loss_pct=7       # e.g 5% stop loss
)
